In [1]:
# import DiCE
import dice_ml
from dice_ml.utils import helpers # helper functions

# Tensorflow libraries
import tensorflow as tf
from tensorflow import keras

In [2]:
import numpy as np
import pandas as pd
from sklearn import datasets, linear_model, preprocessing

In [9]:
names = ['id','thick','size','shape','marg','cell_size','bare',
         'chrom','normal','mit','class']
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/' +
                 'breast-cancer-wisconsin/breast-cancer-wisconsin.data',
                names=names,na_values='?',header=None)

df.drop(columns=['id'],inplace=True)

df = df.dropna()
df.head(6)

# 2 is benign, 4 is malignant

,thick,size,shape,marg,cell_size,bare,chrom,normal,mit,class
0,5,1,1,1,2,1.0,3,1,1,2
1,5,4,4,5,7,10.0,3,2,1,2
2,3,1,1,1,2,2.0,3,1,1,2
3,6,8,8,1,3,4.0,3,7,1,2
4,4,1,1,3,2,1.0,3,1,1,2
5,8,10,10,8,7,10.0,9,7,1,4


In [10]:
df['class']=df['class'].map({4:1,2:0})
df.head(5)

,thick,size,shape,marg,cell_size,bare,chrom,normal,mit,class
0,5,1,1,1,2,1.0,3,1,1,0
1,5,4,4,5,7,10.0,3,2,1,0
2,3,1,1,1,2,2.0,3,1,1,0
3,6,8,8,1,3,4.0,3,7,1,0
4,4,1,1,3,2,1.0,3,1,1,0


In [11]:
df = df.astype(int)
df.head(5)

,thick,size,shape,marg,cell_size,bare,chrom,normal,mit,class
0,5,1,1,1,2,1,3,1,1,0
1,5,4,4,5,7,10,3,2,1,0
2,3,1,1,1,2,2,3,1,1,0
3,6,8,8,1,3,4,3,7,1,0
4,4,1,1,3,2,1,3,1,1,0


In [12]:
d = dice_ml.Data(dataframe=df, continuous_features=[],outcome_name='class')

In [13]:
train, _ = d.split_data(d.normalize_data(d.one_hot_encoded_data))
xtr = train.loc[:, train.columns != 'class']
ytr = train.loc[:, train.columns == 'class']

print(xtr)

     thick_1  thick_2  thick_3  thick_4  thick_5  thick_6  thick_7  thick_8  \
28         0        1        0        0        0        0        0        0   
686        1        0        0        0        0        0        0        0   
614        0        1        0        0        0        0        0        0   
452        0        0        1        0        0        0        0        0   
467        0        0        0        0        0        1        0        0   
..       ...      ...      ...      ...      ...      ...      ...      ...   
421        0        0        0        0        0        0        0        0   
404        1        0        0        0        0        0        0        0   
147        1        0        0        0        0        0        0        0   
248        0        0        0        1        0        0        0        0   
639        0        0        0        0        1        0        0        0   

     thick_9  thick_10  ...  normal_10  mit_1  mit_

In [14]:
#xnames = ['id','thick','size','shape','marg','cell_size','bare',
        # 'chrom','normal','mit']
#x = np.array(df[xnames])

#y=df['class'].values

In [16]:
#splitting the input into training and testing

#from sklearn.model_selection import train_test_split

#xtr, xts, ytr, yts = train_test_split(x,y, test_size=0.2)

In [18]:
#Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
xtr = sc.fit_transform(xtr)
#xts = sc.transform(xts)

In [19]:
#NEURAL NET MODEL

import os
import tensorflow as tf
import tensorflow.keras.backend as K

K.clear_session()

model = tf.keras.Sequential([
  tf.keras.layers.Dense(10, activation='relu', input_shape=(xtr.shape[1],), name='hidden'),
  tf.keras.layers.Dense(1, activation='sigmoid', name='output') #sigmoid output=binary classification
])


from tensorflow.keras import optimizers

opt = optimizers.Adam(lr=0.01) 
model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit( xtr, ytr, batch_size = 3, epochs = 50, shuffle = True)#, validation_data=(xts,yts))

Epoch 1/50
182/182 [==============================] - 0s 1ms/step - loss: 0.1422 - accuracy: 0.9560
Epoch 2/50
182/182 [==============================] - 0s 1ms/step - loss: 0.0332 - accuracy: 0.9872
Epoch 3/50
182/182 [==============================] - 0s 1ms/step - loss: 0.0149 - accuracy: 0.9963
Epoch 4/50
182/182 [==============================] - 0s 1ms/step - loss: 0.0062 - accuracy: 0.9982
Epoch 5/50
182/182 [==============================] - 0s 2ms/step - loss: 0.0042 - accuracy: 1.0000
Epoch 6/50
182/182 [==============================] - 0s 1ms/step - loss: 0.0032 - accuracy: 1.0000
Epoch 7/50
182/182 [==============================] - 0s 1ms/step - loss: 0.0029 - accuracy: 1.0000
Epoch 8/50
182/182 [==============================] - 0s 1ms/step - loss: 0.0028 - accuracy: 1.0000
Epoch 9/50
182/182 [==============================] - 0s 1ms/step - loss: 0.0025 - accuracy: 1.0000
Epoch 10/50
182/182 [==============================] - 0s 1ms/step - loss: 0.0023 - accuracy: 1.0000

In [20]:
backend = 'TF'+tf.__version__[0] # TF2
m = dice_ml.Model(model=model, backend=backend)

In [21]:
# initiate DiCE
exp = dice_ml.Dice(d, m)

In [22]:
# query instance in the form of a dictionary; keys: feature name, values: feature value


query_instance = {'thick':0.88360294, 
                  'size':0.28265077, 
                  'shape':0.25456756, 
                  'marg':0.01051806, 
                  'cell_size':0.34102969,
                  'bare': 1.69443837, 
                  'chrom':1.02372986, 
                  'normal': 45,
                  'mit':-0.35586384}


In [23]:
#print(xtr.shape,xts.shape,ytr.shape,yts.shape)

In [24]:
# generate counterfactuals
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=4, desired_class="opposite")

InvalidArgumentError: Matrix size-incompatible: In[0]: [1,98], In[1]: [89,10] [Op:MatMul]

In [ ]:
widget(annmodel, df1, query_instance)